In [0]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
!pip install -q efficientnet

In [0]:
import numpy as np
import pandas as pd
import cv2
import torch
import torchvision
from google.colab.patches import cv2_imshow
import efficientnet.keras as efn
import tensorflow as tf
import keras
import os,sys
import gc
# from keras_efficientnets import EfficientNetB5,EfficientNetB3
import itertools
from albumentations import (
    CLAHE, Blur,IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur,
     RandomBrightnessContrast, IAASharpen, IAAEmboss, OneOf, Compose, 
     RandomBrightness, RandomContrast, JpegCompression, Resize, ShiftScaleRotate
)

Using TensorFlow backend.


In [0]:
!cp drive/My\ Drive/9d34462453e311ea.zip .

In [0]:
!unzip 9d34462453e311ea.zip 

In [0]:
os.chdir('dataset')

In [0]:
def get_model(weights_=None):
    model_base = keras.applications.DenseNet201(input_shape = (224,224,3),weights=weights_,include_top=False)
    model_base.trainable=False
    model_base._trainable=False
    for layer in model_base.layers:
        layer._trainable = False
    # model._layers.pop(-1)
    modelf = keras.layers.GlobalAveragePooling2D()(model_base.output)
    modelf = keras.layers.Dense(1024, activation='relu')(modelf)
    modelf = keras.layers.Dropout(0.2)(modelf)
    modelf = keras.layers.BatchNormalization()(modelf)
    modelf = keras.layers.Dense(512, activation='relu')(modelf)
    modelf = keras.layers.Dense(4, activation = 'sigmoid')(modelf)
    model = keras.Model(model_base.input,modelf)
    return model

In [0]:
def strong_aug():
    return Compose([
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(var_limit=(1,10)),
        ], p=0.5),
        OneOf([
            MotionBlur(p=0.5),
            MedianBlur(blur_limit=3, p=0.5),
            Blur(blur_limit=2, p=0.5),
        ]),
        RandomBrightness(p=0.3,limit=0.2),
        RandomContrast(p=0.3,limit=0.2),
        JpegCompression(quality_lower=20,p=0.5),
        ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=360, interpolation=1, border_mode=4,always_apply=False, p=0.90)
        ])

In [0]:
def equalize(img):
	img[:,:,0] = cv2.equalizeHist(img[:,:,0])
	return img
def normalize(img):
	img[:,:,0] = ((img[:,:,0]-np.min(img[:,:,0]))*((200-0)/(np.max(img[:,:,0]) - np.min(img[:,:,0]))))
	return img
def hsv(img):
	return cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
def lab(img):
	return cv2.cvtColor(img,cv2.COLOR_BGR2LAB)
def lab2bgr(img):
	return cv2.cvtColor(img,cv2.COLOR_LAB2BGR)
def stretch(img):
    return lab2bgr(equalize(lab(img)))

In [0]:
import copy
from itertools import cycle
batch_size = 26
def generator(img_r, gt_r=None):
    imgc = cycle(img_r)
    gtc = cycle(gt_r)
    x=[]
    y=[]
    for img,gt in zip(imgc,gtc):
            # img = stretch(img)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = (np.reshape(np.array(cv2.resize(img,(224,224),interpolation=cv2.INTER_LINEAR)),(224,224,3))).astype('uint8')
            # img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
            aug = strong_aug()
            img = aug(image=img)['image']
            if np.random.choice([1,2,3,4]) == 4:
                img = cv2.flip(img,1)
            # cv2_imshow(img)
            x.append(img.astype(np.float32)/255)
            y.append(gt)
            if len(x)==batch_size:
                x = np.array(x)
                y = np.array(y)
                k = tuple((x,y))
                x=[]
                y=[]
                yield k

In [0]:
x = cv2.imread('Train Images/'+list(os.listdir('Train Images'))[5])
print(x.shape)

(107, 80, 3)


In [0]:
def get_dataset(batch_size=20):
    train = pd.read_csv('train.csv')
    # train.head()
    classes = list(train['Class'])
    images = list(train['Image'])
    imgs = []
    for i in images:
        imgs.append(cv2.resize(cv2.imread('Train Images/'+i), (80,80)))
    train = pd.concat([train, pd.get_dummies(train['Class'])], axis = 1)
    train = train.drop(columns='Class', axis = 0)
    gts = train[list(train)[1:]]
    train_set = imgs[:5720]
    val_set = imgs[5720:]
    train_gt = np.array(gts.loc[:5719]).astype('float32')
    val_gt = np.array(gts.loc[5720:]).astype('float32')
    train_set = np.array([cv2.resize(i,(80,80)) for i in train_set])
    val_set = np.array([cv2.resize(i,(80,80)) for i in val_set])
    return train,train_set,train_gt,val_set,val_gt

In [0]:
train,train_set,train_gt,val_set,val_gt = get_dataset()

In [0]:
model = get_model('imagenet')
model.compile(optimizer=keras.optimizers.Adam(0.001), 
                loss='categorical_crossentropy',
                metrics=['accuracy'])
train,train_set,train_gt,val_set,val_gt = get_dataset()

In [0]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=1, mode='min', baseline=None, restore_best_weights=False)
mc_callback = keras.callbacks.ModelCheckpoint('../drive/My Drive/model_efn5_he_trained_4.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)
csv_callback = keras.callbacks.CSVLogger('csvlog.log', separator=',', append=False)
reduce_learning_rate = keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)
callback_list = [reduce_learning_rate, mc_callback,es_callback]

In [0]:
model.compile(optimizer=keras.optimizers.Adam(0.0001), 
                loss='categorical_crossentropy',
                metrics=['accuracy'])


In [0]:
# hist = model.fit(train_set,train_gt, validation_data = (val_set,val_gt),verbose=1, batch_size=32,callbacks=callback_list, 
#                  epochs=50,shuffle=True, use_multiprocessing=True,workers=8)

In [0]:
# model = get_model('noisy-student')#keras.models.load_model('../drive/My Drive/model3_he_trained_1.h5')

In [0]:
train_gen = generator(train_set, train_gt)
val_gen = generator(val_set, val_gt)

In [0]:
# model = keras.models.load_model('../drive/My Drive/model_vgg_he_trained_3.h5')

In [0]:
import gc
gc.collect()

1230

In [0]:
hist2 = model.fit_generator(train_gen, steps_per_epoch=len(train_set)//26, epochs = 1000, callbacks=callback_list,
                            verbose=1, validation_data=val_gen, validation_steps = len(val_set)//26,use_multiprocessing=True,workers=8, shuffle=True)

In [0]:
n = next(train_gen)

In [0]:
model.summary()

In [0]:
model = keras.models.load_model('../drive/My Drive/model_efn5_he_trained_4.h5')

In [0]:
test = pd.read_csv('test.csv')

In [0]:
test.shape

(3219, 1)

In [0]:
images = list(test['Image'])
imgs = []
for i in images:
    imgs.append((cv2.cvtColor(cv2.resize(cv2.imread('Test Images/'+i), (224,224)),cv2.COLOR_BGR2RGB).astype(np.float32))/255)

In [0]:
preds = model.predict(np.array(imgs))

In [0]:
ixs = [np.argmax(i) for i in preds ]

In [0]:
mapp = {0:'Attire',1:'Decorationandsignage',2:'Food',3:'misc'}

In [0]:
ixs = [mapp[i] for i in ixs]

In [0]:
test['Class'] = ixs

In [0]:
test.to_csv('out.csv',index=False)